# Sample background 

Individual guano samples were processed in two separate sequencing runs. See Jupyter notebooks titled "bri-P1-notes" and "bri-P2-notes" for initial processing details. This notebook functions by beginning with concatenating individually completed sequencing fastq files, removing any required reads associated for samples within those files we want to discard, then continuing with the clustering of all reads without the mock community. Index-bleed filters and auto-subtract values are assigned to this common pool of clustered reads using the more conservative when applicable. See below for details.

## Other information

- Running core programs:
    - AMPtk 0.8.6
    - vsearch v2.3.4_linux_x86_64
    - usearch usearch v9.2.64_i86linux32
    - python 2.7.6
- Installed the necessary (COI) databases

## Final cleanups before OTU clustering

#### Concatenating
The samples from Project1 and Project2 were combined by concatenating the two files:

In [ ]:
#to concatenate two {}.demux.fq files
cat \
/leo/devon/projects/guano/bri2016/p1_data/bri-chunk1/cleaned.demux.fq \
/leo/devon/projects/guano/bri2016/p2_data/bridropd.demux.fq > prebriall.demux.fq

#### Sample read removal
Initially when completing the analyses on the individual sequencing runs, samples in Project1 with more than 5,000 reads were kept while samples in Project2 were kept with 10,000 reads (for various reasons including NTC read depth, mock community contamination, etc.) I wanted to be unifrom as possible when combining the two datasets and went with a more conservative approach by requiring 10,000 reads for any sample in this project. Therefore, 28 additional samples from Project1 were dropped, as listed in the file **morefiles2drop.txt**. 

In [ ]:
amptk remove -i prebriall.demux.fq -f morefiles2drop.txt -o briall.demux.fq

The total number of samples following concatenation between both projects is **210 samples**. We are highly confident that these samples contain a meaningful number of reads to ensure that our community analyses are robust, while also containing as little type-1 error as possible in the forthcoming OTU table.

## part 2 - OTU clustering

In [ ]:
#to cluster P1L1 data:
amptk dada2 \
--fastq briall.demux.fq \
--out briall \
--length 180 \
--platform illumina \
--uchime_ref COI

Here's the output using the default settings with DADA2:

Next up is to filter the OTU table with the fasta file listed above:

# Part 3: filtering OTU table

Note that there is something a bit unique about this analysis: the two independent sequencing runs (P1 and P2) each had their own mock communities; normally I'd take the more conservative estimate from either run and then apply that here, but there were some subtleties that required keeping the mock community in this dataset. These are going to be analyzed now to determine a few factors:  

1. What's the **index-bleed** threshold we want to apply? 
    Earlier work from the independent runs suggested somewhere between 0.1 - 0.3%

2. What's the **subtract** threshold we want to apply?
    Independent sequencing runs reported values of 47 and 86; thus the more conservative measure would be 86. However, this higher value (from Project 1 sequencing run) allowed for three non-intended OTUs to remain in the dataset in low values; it's unclear whether these are the result of index-bleed from actual *BRI samples*, or from another sample placed on the same P1 lane (for example, from O'Rourke samples, or others), or if it's from low-level contamination that was incorporated at the PCR step earlier in the workflow.  

3. How many OTUs will be dropped if necessary to balance the **subtract** value with overall OTU retention?  

To address these questions we'll run preliminary filtering steps and assess their output, as described below.

In [ ]:
amptk filter \
-i briall.cluster.otu_table.txt \
-f briall.cluster.otus.fa \
-b mockIM3 \
--keep_mock \
--calculate in \
--mc /leo/devon/projects/guano/mock-fa/CFMR_insect_mock3.fa \
--debug \
--subtract auto \
-o testfilt

And the relevant output:

In other words, we have a pretty high index bleed (but not awful), but a HUGE **subtract** value. From earlier work on each sequencing runs (P1 and P2), this is likely due to a handful of samples. To address which samples contain the highest number of reads, see the basic notes outlined in "bri-P2-notes". Bioinformatic steps and subsequent results are briefly explained below.  

First, identifying the number of contaminant OTUs in our combined mock community:

In [ ]:
sed -i 's/#OTU ID/OTUid/' testfilt.final.txt
sed -i 's/#OTU ID/OTUid/' testfilt.normalized.num.txt
awk -F '\t' '{print NF; exit}' testfilt.final.txt
    #212

cut testfilt.normalized.num.txt -f 1,211 | sort -k2,2n | awk '$2 != "0.0" {print $0}'
    #OTU249 is the culprit

The above script produces a long list (see below) of all OTUs present in our mock community, though nearly all of these unwanted OTUs are in very low abundance (while, importantly, all the OTUs that should be in our mock community are there in relatively high numbers). Note, however, one OTU (OTU249) contains a much higher number of reads than any other unwanted OTU. This is in one sense reassuring, as it implies that the high **subtract** value found in our first-pass filtering step is due to just a single OTU, and by dropping just that one OTU we sould be able to retain far more reads/OTUs when our final filter is applied.  

- ...*(more values in list containing OTUs with fewer than 36.0 reads)*...
- OTU497	36.0
- OTU151	41.0
- OTU21	48.0
- MockIM34_pident=100.0_OTU349	682.0
- **OTU249	977.0**
- MockIM49_pident=100.0_OTU207	3960.0
- MockIM6_pident=100.0_OTU188	4976.0
- MockIM15_pident=99.4_OTU161	7036.0  

The next question is whether or not **OTU249** persists among many samples in addition to our mock community:

In [ ]:
grep "\\bOTU249\\b" testfilt.normalized.num.txt

The result indicates that OTU249 is highest in our mock community (recall it's value was **977** reads); only 14 other samples have more than 100 reads (most are 0.0). Average was ~31, but deviation was about 119. See **otu249counts.txt** for table of these counts. I further investigated whether or not there was any relationship for a particular factor (index, PCR or DNA plate/well, etc.) associated with contamination; there doesn't appear to be one among the most contaminated. See **otu249contamtable.txt** for these results.  

To ensure this OTU is likely the same contaminant observed in individual analyses performed earlier, a BLAST search was performed: 

In [ ]:
grep "\\bOTU249\\b" briall.cluster.otus.fa -A 3

Alignment results indicate it's Fannia canicularis, the Lesser housefly, which was what was observed from earlier independent analyses for the P1 data set. It's unclear whether contamination was from index bleed from other samples not part of this dataset (ex. other bird sequencing projects), or from primer contamination. Regardless, dropping this one OTU is likely of little consequence to broader interpretation as it is not particularly common in our dataset.  

To remove OTU249:

In [ ]:
amptk drop \
-i briall.cluster.otus.fa \
-r briall.demux.fq \
-l OTU249 \
-o bri

The above steps likely address the problem of OTU contamination in our mock community. However, there is a single negative control in this dataset that should be dropped - sample "**negbri-41C03**". This sample wasn't removed initially because it's useful to notice how many OTUs are present, and to what degree (in terms of relative/normalized reads). In certain cases where negative controls show some appreciable number of amidst true samples, we find that a single OTU or two should be dropped from all samples (like above). However, this sample doesn't appear to be a typical cross-contamination event; rather, I think this sample represents one of two scenarios:  
- (a) A guano sample placed in an adjacent well split and fell into this negative control well, and thus represents a pseudoreplicate.
- (b) A guano sample was misplaced directly into the negative control well, and thus represents a true sample that is simply mislabeled.  

A quick analysis of this sample suggests the following:  
1. There is a single OTU that contains the majority of the reads associated with this sample: OTU2 (see file **negconcounts.txt** for all OTU counts associated with this sample). This OTU exists in just a handful of all true samples, with only 7 samples containing >100 reads (though 5 samples contain >10,000 reads). Thus this OTU best aligns to one of two *crane flys*, which are fairly large and may take up a significant proportion of a single guano sample.
2. There are just 5 other OTUs that contain >100 reads:
    - OTU48, OTU154, OTU34 follow the similar pattern above: only a few samples contain many reads associated with this particular OTU, and there's a huge variance in total number of reads. 
    - OTU48 is another crane fly; OTU154 is a caddisfly; OTU34 is a northeastern moth; OTU150 is a leafroller moth. In other words, this sample contains insects we'd expect in a piece of bat guano (so you can rule out contamination from other guano, say bird guano, from other projects).
3. I created another file, **contamprobs.txt** that looks at these top 6 OTUs for all samples. It doesn't fit any single other sample perfectly, but it matches to loads of other samples qualitatively. In other words, plenty of samples contain some number of reads for these OTUs in question, but no single sample has at least 100 reads for every one of these OTUs.  

Observation #3 above strongly suggests this being a true sample, though it's unclear whether its a case of a guano sample splitting into the wrong well (as in '**a**') or a true sample that was mislabeled (as in '**b**'). Dropping the OTUs associated with this sample is likely unnecessary. Rather, dropping this sample from further analysis is what makes more sense, as we'd like to retain these OTUs that are present in the data.  

This will be done post-taxonomic classification in R, however, if this information was available a priori we could have dropped it earlier with the ** *amptk remove* ** step.

We'll re-run our filtering program this time without that contaminant OTU:

In [ ]:
amptk filter \
-i bri.cleaned.otu_table.txt \
-f bri.cleaned.otus.fa \
-b mockIM3 \
--keep_mock \
--calculate in \
--mc /leo/devon/projects/guano/mock-fa/CFMR_insect_mock3.fa \
--debug \
--subtract auto \
-o testfilt2

The results are as follows:

Results indicate that our **index-bleed** can be set at **0.05%** (a typical value) and the auto-subtract value can be set to **48**. Note that while this is less of a conservative of a subtract value for one of the two runs (P2 was **47** while P1 was **86**), the fact that we summed up *three* mock communities is potentially artificially inflating our overall counts for any OTU associated with a mock community (despite that fact that we're normalizing reads when making these calculations).

Notably, we've increased the number of OTUs retained from our initial filtering (default) parameters when including OTU249. By dropping that single OTU from the dataset we've reduced the **subtract** value by about 900, thus we've recovered the number of OTUs by more than double (we kept **984 OTUs** rather than the earlier **406**).  

A final filtering step is applied whereby the mock community reads are removed leaving us with just true samples (and that unknown sample masquerading as a negative control sample).

In [ ]:
amptk filter \
-i bri.cleaned.otu_table.txt \
-f bri.cleaned.otus.fa \
-b mockIM3 \
--calculate in \
--mc /leo/devon/projects/guano/mock-fa/CFMR_insect_mock3.fa \
--subtract auto \
-o bri

The arguments above will produce the following files:  
- **filtd.filtered.otus.fa** is the final filtered fasta file (say that five times fast)
- **filtd.final.binary.txt** is the presence/absence OTU table after filtering
- **filtd.stats.txt** is the number of OTUs in each sample before and after filtering
- **filtd.final.txt** is the OTU table with normalized read counts (noramlized to the number of reads in each sample)
- **filtd.amptk-filter.log**  

The next step is to assign taxonomy by using the **bri.filtered.otus.fa** and **bri.final.binary.txt** files. 

## Part 4: Assigning Taxonomy to OTUs

Make sure to have already downloaded the necessary database (COI). Need to run this command for every instance in which you've generated a uniquely filtered dataset from Parts 1-3. Here's just a pair of examples you'd run in their respective directories: